In [1]:
import pandas as pd
from lxml import etree
import os
import html2text
import re
import numpy as np

In [2]:
path = r'C:\Users\chenmeng\Documents\FDDC_2\dingzeng_train\7880.html'
f = open(path,'r',encoding='utf-8')
text = f.read()
f.close()

In [47]:
html = etree.HTML(text)
html.xpath('//tbody')


[<Element tbody at 0x1f634ef8048>,
 <Element tbody at 0x1f634f64cc8>,
 <Element tbody at 0x1f634f64208>,
 <Element tbody at 0x1f634f644c8>,
 <Element tbody at 0x1f634f64348>,
 <Element tbody at 0x1f634f64f48>,
 <Element tbody at 0x1f634f64388>,
 <Element tbody at 0x1f634f64288>,
 <Element tbody at 0x1f634f64c88>,
 <Element tbody at 0x1f634f64448>,
 <Element tbody at 0x1f634f64308>,
 <Element tbody at 0x1f634f642c8>,
 <Element tbody at 0x1f634f64dc8>,
 <Element tbody at 0x1f634f64d08>,
 <Element tbody at 0x1f634f64e48>,
 <Element tbody at 0x1f634f24488>,
 <Element tbody at 0x1f634f244c8>,
 <Element tbody at 0x1f634f24688>,
 <Element tbody at 0x1f634f24748>,
 <Element tbody at 0x1f634f24908>,
 <Element tbody at 0x1f634f24448>,
 <Element tbody at 0x1f634f249c8>,
 <Element tbody at 0x1f634f24b08>,
 <Element tbody at 0x1f634f243c8>,
 <Element tbody at 0x1f634f245c8>,
 <Element tbody at 0x1f634f24888>,
 <Element tbody at 0x1f634f24408>,
 <Element tbody at 0x1f634f24a08>,
 <Element tbody at 0

In [40]:
tbody = html.xpath('//tbody')[3]
r = tbody.xpath('tr')
r[0].xpath('td')

[<Element td at 0x1f634f1c4c8>,
 <Element td at 0x1f634f1c3c8>,
 <Element td at 0x1f634f1c588>,
 <Element td at 0x1f634f1c5c8>]

In [38]:
class TableParser:
    def __init__(self):
        self.rows = 0
        self.columns = 0
        self.tablehead = ''
        self.table = []
        self.tabletype = 0
        
    def parse(self,tbody):
        r = tbody.xpath('tr')
        self.rows = len(r)
        c = [len(r[i].xpath('td')) for i in range(self.rows)]
        self.columns = max(c)
        table = np.empty((self.rows,self.columns),dtype='<U100')
        
        for i in range(self.rows):
            c = r[i].xpath('td')
            for j in range(len(c)):
                rowspan = c[j].xpath('@rowspan')
                colspan = c[j].xpath('@colspan')
                text = ''.join(c[j].xpath('text()'))
                t = self.clean(text)
                
                if (not rowspan and not colspan):
                    for k in range(self.columns):
                        if not table[i][k]:
                            table[i][k] = t
                            break
                if rowspan:
                    span = int(rowspan[0])
                    for k in range(self.columns):
                        if not table[i][k]:
                            for m in range(span):
                                table[i+m][k] = t
                            break
                if colspan:
                    span = int(colspan[0])
                    count = 0
                    for k in range(self.columns):
                        if not table[i][k]:
                            table[i][k] = t
                            count += 1
                            if count == span:
                                break
        self.table = table
    
    def get_tablehead(self,tbody):
        s = tbody.xpath('tr[1]/td//text()')
        s = ''.join(s)
        s = self.clean(s)
        self.tablehead = s
        return s
    
    def clean(self,s):
        s = s.replace('\n','')
        s = s.replace(' ','')
        return s

In [39]:
tableparser = TableParser()

In [41]:
tableparser.parse(tbody)

In [48]:
tableparser.table

array([['序号', '认购对象', '配售股数（股）', '配售金额（元）'],
       ['1', '广州海印实业集团有限公司', '29,160,239', '243,487,995.65'],
       ['2', '太平洋资产管理有限责任公司', '6,000,000', '50,100,000.00'],
       ['3', '民生加银基金管理有限公司', '11,520,000', '96,192,000.00'],
       ['4', '东海证券股份有限公司', '12,000,000', '100,200,000.00'],
       ['5', '融通基金管理有限公司', '7,000,000', '58,450,000.00'],
       ['6', '平安大华基金管理有限公司', '28,200,000', '235,470,000.00'],
       ['7', '深圳市吉富启瑞投资合伙企业（有限合伙）', '6,000,000', '50,100,000.00'],
       ['合计', '99,880,239', '833,999,995.65', '']],
      dtype='<U100')

In [132]:
r = tbody.xpath('tr')
c = [len(r[i].xpath('td')) for i in range(4)]

In [133]:
c

[6, 6, 4, 5]

In [129]:
c

[0, 0, 0, 0]

In [182]:
c[0].xpath('text()')

['\n      股东\n        名称\n     ']

In [222]:
table = np.empty((4,6),dtype='<U100')

In [223]:
for i in range(4):
    c = r[i].xpath('td')
    for j in range(len(c)):
        rowspan = c[j].xpath('@rowspan')
        colspan = c[j].xpath('@colspan')
        text = ''.join(c[j].xpath('text()'))
        t = text.replace('\n','').replace(' ','')
        if (not rowspan and not colspan):
            for k in range(6):
                if not table[i][k]:
                    table[i][k] = t
                    break
        
        if rowspan:
            span = int(rowspan[0])
            for k in range(6):
                if not table[i][k]:
                    for m in range(span):
                        table[i+m][k] = t
                    break
        if colspan:
            span = int(colspan[0])
            count = 0
            for k in range(6):
                if not table[i][k]:
                    table[i][k] = t
                    count += 1
                    if count == span:
                        break
        

In [224]:
table

array([['股东名称', '减持方式', '减持期间', '减持均价（元）', '减持股数（股）', '减持比例（%）'],
       ['宁波睿勇投资有限公司', '大宗交易', '2014年5月27日', '9.32', '1,950,000', '1.46'],
       ['宁波睿勇投资有限公司', '大宗交易', '2014年5月28日', '9.30', '1,940,000', '1.44'],
       ['宁波睿勇投资有限公司', '合计', '-', '-', '3,890,000', '2.90']],
      dtype='<U100')

In [203]:
np.typeDict

{'?': numpy.bool_,
 0: numpy.bool_,
 'byte': numpy.int8,
 'b': numpy.int8,
 1: numpy.int8,
 'ubyte': numpy.uint8,
 'B': numpy.uint8,
 2: numpy.uint8,
 'short': numpy.int16,
 'h': numpy.int16,
 3: numpy.int16,
 'ushort': numpy.uint16,
 'H': numpy.uint16,
 4: numpy.uint16,
 'i': numpy.int32,
 5: numpy.int32,
 'uint': numpy.uint32,
 'I': numpy.uint32,
 6: numpy.uint32,
 'intp': numpy.int64,
 'p': numpy.int64,
 9: numpy.int64,
 'uintp': numpy.uint64,
 'P': numpy.uint64,
 10: numpy.uint64,
 'long': numpy.int32,
 'l': numpy.int32,
 7: numpy.int32,
 'L': numpy.uint32,
 8: numpy.uint32,
 'longlong': numpy.int64,
 'q': numpy.int64,
 'ulonglong': numpy.uint64,
 'Q': numpy.uint64,
 'half': numpy.float16,
 'e': numpy.float16,
 23: numpy.float16,
 'f': numpy.float32,
 11: numpy.float32,
 'double': numpy.float64,
 'd': numpy.float64,
 12: numpy.float64,
 'longdouble': numpy.float64,
 'g': numpy.float64,
 13: numpy.float64,
 'cfloat': numpy.complex128,
 'F': numpy.complex64,
 14: numpy.complex64,
 'c

In [231]:
table = [['']*6]*4
table

[['', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', '', '']]

In [233]:
table[0][0] = 1
table

[[1, '', '', '', '', ''],
 [1, '', '', '', '', ''],
 [1, '', '', '', '', ''],
 [1, '', '', '', '', '']]